# Part 2 Fondamental Value Modeling

### Import

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint


## Cointegration

In [ ]:

# Example: price and div are pandas Series with DatetimeIndex
# price: Nasdaq index level
# div:   dividend amount (same date index)

df = pd.concat([price.rename("P"), div.rename("D")], axis=1).dropna()

# OPTIONAL: go to monthly
# price → last value of month, dividends → sum over month
df_m = pd.DataFrame({
    "P": df["P"].resample("M").last(),
    "D": df["D"].resample("M").sum()
}).dropna()


In [ ]:
df_m = df_m[df_m["P"] > 0]      # ensure positivity
df_m = df_m[df_m["D"] > 0]

df_m["p"] = np.log(df_m["P"])
df_m["d"] = np.log(df_m["D"])

# Campbell's log dividend–price ratio
df_m["dp"] = df_m["d"] - df_m["p"]


In [ ]:


p = df_m["p"]
d = df_m["d"]

# --- Step 1: long-run regression p_t = α + β d_t + u_t ---
X = sm.add_constant(d)
ols_res = sm.OLS(p, X).fit()
print(ols_res.summary())

# residuals are the estimated cointegrating error û_t
u_hat = ols_res.resid

# --- Step 2: ADF test on residuals (Engle–Granger) ---
adf_stat, adf_p, _, _, crit_vals, _ = adfuller(u_hat, maxlag=12, regression='c')

print("\nEngle–Granger residual ADF test:")
print(f"ADF statistic: {adf_stat:.3f}")
print(f"p-value      : {adf_p:.3f}")
print("Critical values:", crit_vals)


In [ ]:
adf_dp = adfuller(df_m["dp"], maxlag=12, regression='c')
dp_stat, dp_p, _, _, dp_crit, _ = adf_dp

print("\nADF test on dp_t = log(D) - log(P):")
print(f"ADF statistic: {dp_stat:.3f}")
print(f"p-value      : {dp_p:.3f}")
print("Critical values:", dp_crit)
